In [5]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Part 1

In [33]:
from copy import deepcopy

class Rock:
    def __init__(self, positions, final_pos):
        max_y = max(final_pos.keys())
        min_x = min([ i[0] for i in positions])
        shift_right = 2-min_x
        shift_up = max_y+4
        
        self.pos = [ (i[0]+shift_right, i[1]+shift_up) for i in positions]
        
    
    def move_down(self, final_pos):
        new_pos = []
        reject=False
        for x in self.pos:
            nx = [x[0], x[1]-1]
            if nx[1] in final_pos.keys() and nx[0] in final_pos[nx[1]]:
                reject =True
                break
            new_pos.append(nx)
        if reject:
            for x in self.pos:
                try:
                    final_pos[x[1]].append(x[0])
                except:
                    final_pos[x[1]] = [x[0]]
                    
            return True, final_pos
        
        self.pos = new_pos
        return False, final_pos
                
    def move_jet(self,jet,final_pos):
        push = -1 if jet == '<' else 1 
        new_pos = []
        reject=False
        for x in self.pos:
            nx = [x[0]+push, x[1]]
            if nx[0] <0 or nx[0] > 6:
                reject =True
                break
            if nx[1] in final_pos.keys() and nx[0] in final_pos[nx[1]]:
                reject =True
                break
            new_pos.append(nx)
            
        if reject:
            return 
        self.pos = new_pos
                

In [34]:
rocks = [  [(i,0) for i in range(4)],
           [(1,2),(0,1),(1,1),(2,1),(1,0)],
           [(0,0),(1,0),(2,0),(2,1),(2,2)],
           [(0,i) for i in range(4)],
           [(0,0),(1,0),(0,1),(1,1)]
        ]
         

In [35]:
def read_jetstream(filename):
    with open(filename,'r') as f:
        line = f.readline()
        line = line.strip()
        return line
        
    

In [45]:
def run_part_one(filename):
    jets = read_jetstream(filename)
    ljets = len(jets)
    nrocks = 0 
    ij=0
    
    rock_settled = True 
    settled={ 0 : [ i for i in range(7)]}
    
    while nrocks < 2022:
        if rock_settled :
            rock = Rock(rocks[nrocks%5],settled)
            rock_settled = False
        while not rock_settled :
            rock.move_jet(jets[ij%ljets], settled)
            ij+=1
            rock_settled, settled =rock.move_down( settled)
        nrocks+=1
        
                        
            
    return max(settled.keys())
    

In [46]:
%%time
run_part_one('input/test_17.txt')

CPU times: user 98 ms, sys: 2.59 ms, total: 101 ms
Wall time: 100 ms


3068

In [47]:
%%time
run_part_one('input/day_17.txt')

CPU times: user 97.2 ms, sys: 3.06 ms, total: 100 ms
Wall time: 99.7 ms


3059

# Part 2

In [193]:
def run_part_two(filename,trocks= 1000000000000, hstr=20, start=35,above=15):
    jets = read_jetstream(filename)
    ljets = len(jets)
    nrocks = 0 
    ij=0
    
    rock_settled = True 
    settled={ 0 : [ i for i in range(7)]}
   
    low_level = 0 
    seen={}
    prev=0
    start_level=None
    while nrocks <trocks:
        if rock_settled :
            rock = Rock(rocks[nrocks%5],settled)
            rock_settled = False
        while not rock_settled :
            rock.move_jet(jets[ij%ljets], settled)
            ij+=1
            rock_settled, settled =rock.move_down( settled)
        nrocks+=1
        
        level= max(settled.keys()) 
        if nrocks%start == 0 and  nrocks>1:
            print(level -prev)
            prev=level
        if nrocks == start:
            start_level = level
        if  nrocks == start + above:
            print (f'add {level-start_level}')
        
        if nrocks%start == 0 and  nrocks>1 and nrocks/start > 5 :
            return
            
        if False:# level > hstr:
            hash_str = ''
            for i in range(hstr):
                blank = list('.......')
                for r in  settled[level-i]:
                    blank[r]='#'
                hash_str+=''.join(blank)
            state = (nrocks%5, ij%ljets, hash_str)
            if state in seen.keys():
                print (seen[state], level, nrocks, nrocks-seen[state][1]) 
                return nrocks-seen[state][1]
#               break
            else:
                seen[state] = (level, nrocks)
                        
            
    return level
    

In [200]:
def run_part_two_find_repeat(filename,trocks= 1000000, hstr=20, ):
    jets = read_jetstream(filename)
    ljets = len(jets)
    nrocks = 0 
    ij=0
    
    rock_settled = True 
    settled={ 0 : [ i for i in range(7)]}
   
    low_level = 0 
    seen={}
    while nrocks <trocks:
        if rock_settled :
            rock = Rock(rocks[nrocks%5],settled)
            rock_settled = False
        while not rock_settled :
            rock.move_jet(jets[ij%ljets], settled)
            ij+=1
            rock_settled, settled =rock.move_down( settled)
        nrocks+=1
        
        level= max(settled.keys()) 
        if  level > hstr:
            hash_str = ''
            for i in range(hstr):
                blank = list('.......')
                for r in  settled[level-i]:
                    blank[r]='#'
                hash_str+=''.join(blank)
            state = (nrocks%5, ij%ljets, hash_str)
            if state in seen.keys():
                print (seen[state], level, nrocks, nrocks-seen[state][1]) 
                return nrocks-seen[state][1]
            else:
                seen[state] = (level, nrocks)
                        
            
    return level
    

In [201]:
(1e12//35 - 1)*53+60 

1514285714270.0

In [202]:
1e12%35

15.0

In [203]:
%%time
run_part_two('input/test_17.txt',10005, hstr=20)

60
add 18
53
53
53
53
53
CPU times: user 13.3 ms, sys: 2.63 ms, total: 15.9 ms
Wall time: 14.5 ms


In [204]:
%%time
run_part_two('input/test_17.txt',205, hstr=20)

60
add 18
53
53
53
53
CPU times: user 13.3 ms, sys: 2.4 ms, total: 15.8 ms
Wall time: 14.1 ms


316

In [209]:
run_part_two_find_repeat('input/day_17.txt',20000000, hstr=100)

(549, 347) 3123 2062 1715


1715

In [206]:
1e12%1715

15.0

In [207]:
%%time
run_part_two('input/day_17.txt',100005, hstr=100,start=1715,above=15)

2592
add 23
2574
2574
2574
2574
2574
CPU times: user 2.08 s, sys: 5.48 ms, total: 2.08 s
Wall time: 2.08 s


In [208]:
(1e12//1715 - 1)*2574+2592 +23

1500874635587.0